# Greedy Opponent
* Calculate the state values of a random strategy against a random opponent.  
* Use state values to implememnt a greedy one-step look ahead policy.

In [1]:
import itertools
import numpy as np
from IPython.display import clear_output
from sys import exit
from copy import deepcopy
from collections import defaultdict
from random import choice

## Utility Functions


In [2]:
# check if a value is an integer
def is_int(value):
    try: int(value);   return True
    except: return False

# repalce special characters
def replace_special(s, empty_piece=' '):
    return s.replace('-3', '?').replace('-2', '#').replace('-1', empty_piece)

In [3]:
class game(object):
    def __init__(self, w, h, n, init_board=None, board_wrap=False):
        self.player = 0
        self.status = 'active'

        # set up initial board
        if init_board is not None: 
            self.board = init_board
            h, w = init_board.shape
        else: self.board = np.array([[-1]*w]*h)
        self.init_board = deepcopy(self.board)
        self.w, self.h, self.n = w, h, n
        self.col_count = [next((h-k-1 for k in range(h)[::-1] if col[k] >-2), h) for col in self.board.T]
        self.col_capacity = [sum(col>-2) for col in self.board.T]
        self.feasible_moves = [k for k in range(w) if self.col_count[k]<self.col_capacity[k]]
        
        # modify for board wrap
        self.board_wrap = board_wrap

        # add rows to top and bottom
        bot = np.array([self.board[h-1,:]]) if board_wrap else np.array([[-1]*w])
        top = np.array([self.board[0,:]]) if board_wrap else np.array([[-1]*w])
        self.board = np.concatenate((bot, self.board, top), axis=0)
            
        # add cols to left and right
        left = np.array([self.board[:,0]]).T if board_wrap else np.array([[-1]*(h+2)]).T
        right = np.array([self.board[:,w-1]]).T if board_wrap else np.array([[-1]*(h+2)]).T
        self.board = np.concatenate((right, self.board, left), axis=1)
        
        # save a copy of the board for game resets
        self.init_full_board = deepcopy(self.board)
        

    def reset_game(self):
        self.player = 0
        self.status = 'active'
        self.col_count = [next((self.h-k-1 for k in range(self.h)[::-1] if col[k] >-2), self.h) for col in self.init_board.T]
        self.col_capacity = [sum(col>-2) for col in self.init_board.T]
        self.feasible_moves = [k for k in range(self.w) if self.col_count[k]<self.col_capacity[k]]
        self.board = deepcopy(self.init_full_board)

        
    # display the status of the board
    def print_board(self):
        clear_output() # comment out for debugging
        print('Connect-' + str(self.n))
        if self.board_wrap:
            # print top margin
            lcorner = replace_special(str(self.board[0,0]))
            rcorner = replace_special(str(self.board[0,self.w+1]))
            row = ''.join('(' + str(r) + ')' for r in self.board[0,1:self.w+1])
            row = replace_special(row)
            print('(' + lcorner + ')' + row + '(' + rcorner + ')')

            # extract left and right margins 
            lmargin = [replace_special(str(num)) for num in self.board[1:self.h+1,0]]
            rmargin = [replace_special(str(num)) for num in self.board[1:self.h+1,self.w+1]]
            
            # print left and right margins with actual board
            for k in range(self.h):
                row = '  '.join(str(r) for r in self.board[k+1,1:self.w+1])
                row = replace_special(row, '.')
                print('(' + lmargin[k] + ') ' + row + ' (' + rmargin[k] + ')')
                
            # print bottom margin
            lcorner = replace_special(str(self.board[self.h+1,0]))
            rcorner = replace_special(str(self.board[self.h+1,self.w+1]))
            row = ''.join('(' + str(r) + ')' for r in self.board[self.h+1,1:self.w+1])
            row = replace_special(row)
            print('(' + lcorner + ')' + row + '(' + rcorner + ')')
            
            # print column numbers
            print('   ' + '-'*(3*self.w))
            print('    ' + '  '.join(str(col) for col in range(self.w)))
            
        else:
            for k in range(self.h):
                row = ' '.join(str(r) for r in self.board[k+1,1:self.w+1])
                row = replace_special(row, '.')
                print('| ' + row + ' |')
            print('-'*(3+2*self.w))
            print('  ' + ' '.join(str(col) for col in range(self.w)))                    


    # get user's input
    def col_input(self):
        # ask for and validate input
        while True:
            col = input('Player ' + str(self.player) + ': Enter the column to place your piece. ')
            if col.lower()=='q': print('Thanks for playing!');   exit(0)
            if col=='`': col = 0 # define an alias for 0

            # check for valid input
            if is_int(col): 
                col = int(col)
                if col>=0 and col<self.w and col in self.feasible_moves: break
        return col

    # count connections to the left and right of pos
    def cnct_gt_n(self, vec, pos): 
        count, player = 1, vec[pos]
        
        # check if at least n in length
        if sum(v==player or v==-3 for v in vec) < self.n: return False
        
        for v in vec[:pos][::-1]:
            if v==player or v==-3: count += 1
            else: break
        for v in vec[pos+1:]:
            if v==player or v==-3: count += 1
            else: break
        return count >= self.n

    # extract backward diagonal
    def bdiag(self, board, shift=0):
        h, w = self.h+2, self.w+2
        if shift==0: return [board[k, w-k-1] for k in range(min(h, w))]
        if shift>0: return [board[k, w-(k+shift)-1] for k in range(min(h, w-shift))]
        return [board[k-shift, w-k-1] for k in range(min(h+shift, w))]   

    # check intersecting row, col, diagonals for connect-n
    def connect_n_check(self, row, col):
        forward_diag = np.diag(self.board, col-row)
        backward_diag = self.bdiag(self.board, self.w+2-1-col-row)
        return (self.cnct_gt_n(self.board[:, col], row) or               # col
                self.cnct_gt_n(self.board[row, :], col) or               # row
                self.cnct_gt_n(forward_diag, min(row, col)) or           # fdiag
                self.cnct_gt_n(backward_diag, min(row, self.w+2-1-col))) # bdiag
    
    # update game        
    def update_game(self, col):
        # update col_count
        self.col_count[col] += 1
        
        # skip special pieces
        row = self.h-self.col_count[col]
        while self.board[row+1, col+1]<-1: 
            row -= 1 
            self.col_count[col] += 1
                    
        # update board
        self.board[row+1, col+1] = self.player # assign move to player
        if self.board_wrap:
            if row==0: 
                self.board[self.h+1, col+1] = self.player # fill bot margin
                if col==self.w-1: self.board[self.h+1, 0] = self.player # fill bot left corner
                elif col==0: self.board[self.h+1, self.w+1] = self.player # fill bot right corner
            if row==self.h-1: 
                self.board[0, col+1] = self.player # fill top margin
                if col==self.w-1: self.board[0, 0] = self.player # fill top left corner
                elif col==0: self.board[0, self.w+1] = self.player # fill top right corner
            if col==0: self.board[row+1, self.w+1] = self.player # right
            if col==self.w-1: self.board[row+1, 0] = self.player # left
            
        # update feasible moves
        if self.col_count[col]>=self.col_capacity[col]: self.feasible_moves.remove(col)
                    
        # check for win
        if self.connect_n_check(row+1, col+1): self.status = 'won';   return
        if self.board_wrap:        
            if row==0: 
                if self.connect_n_check(self.h+1, col+1): self.status = 'won';   return
                if col==self.w-1 and self.connect_n_check(self.h+1, 0): self.status = 'won';   return
                if col==0 and self.connect_n_check(self.h+1, self.w+1): self.status = 'won';   return
            if row==self.h-1: 
                if self.connect_n_check(0, col+1): self.status = 'won';   return
                if col==self.w-1 and self.connect_n_check(0, 0): self.status = 'won';   return
                if col==0 and self.connect_n_check(0, self.w+1): self.status = 'won';   return
            if col==0 and self.connect_n_check(row+1, self.w+1): self.status = 'won';   return            
            if col==self.w-1 and self.connect_n_check(row+1, 0): self.status = 'won';   return
        
        # check for draw
        if not self.feasible_moves: self.status = 'draw';   return
        
        # switch players
        self.player = 1*(self.player==0)
        
    # two-player game
    def two_player_game(self):
        while True:
            self.print_board()
            col = self.col_input()
            self.update_game(col)
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('Player ' + str(self.player) + ' won!')
                else: print('Game is a draw.')
                return     
w, h, n = 5, 3, 3
init_board = np.array([[-1]*w]*h)
init_board[1,2] = -2
#init_board[0,2] = -3

g = game(w, h, n, init_board, board_wrap=False)
g.two_player_game()

Connect-3
| . . . . . |
| . . # . . |
| . . . . . |
-------------
  0 1 2 3 4
Player 0: Enter the column to place your piece. q
Thanks for playing!


SystemExit: 0

To exit: use 'exit', 'quit', or Ctrl-D.


In [4]:
class greedy_opponent(game):
    def __init__(self, w, h, n, init_board=None, board_wrap=False):
        game.__init__(self, w, h, n, init_board, board_wrap)
        self.traj = ''
        self.state_value = defaultdict(lambda: 0)
        self.state_count = defaultdict(lambda: 0)
        
        self.temp_value = defaultdict(lambda: 0)
        
    def reset_game(self):
        game.reset_game(self)
        self.traj = ''
        
    def update_game(self, col):
        game.update_game(self, col)
        self.traj += str(col) 
        
    def update_state_values(self):
        N = sum(self.col_count)
        winlose = 1*(self.status=='won')
        for k in range(1,N+1)[::-1]:
            key = self.traj[:k]
            cnt = self.state_count[key]+1
            val = self.state_value[key]
            self.state_count[key] = cnt
            self.state_value[key] = (winlose + (cnt-1)*val)/cnt
            winlose *= -1
            
    # random move
    def random_move(self): return choice(self.feasible_moves)
    
    # move to the state with best value
    def greedy_move(self):
        state = self.traj
        bst_val, bst_mv = -2, -1
        for mv in self.feasible_moves:
            val = self.temp_value[state+str(mv)]
            if val > bst_val: bst_val, bst_mv = val, mv
        return bst_mv
    
    # play a given policy (both players)
    def value_iteration(self, policy, n_games=1):
        for k_game in range(n_games):
            self.reset_game()
            while self.status=='active':
                col = policy()
                self.update_game(col)      
            self.update_state_values()        
            
    def play_game(self, first_move=True):
        self.reset_game()
        if first_move:
            self.print_board()
            col = self.col_input()
            self.update_game(col)
        else: self.player = 1
        
        while True:
            # opponent's greedy move
            self.print_board()
            col = self.greedy_move()
            self.update_game(col)      
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('You lost.')
                break            
            # your move
            self.print_board()
            col = self.col_input()
            self.update_game(col) 
            if self.status!='active':
                self.print_board()
                if self.status=='won': print('You won!')
                break
        if self.status=='draw': print('The game is a draw.')
        #self.update_state_values()        
        
w, h, n = 4, 3, 3
init_board = np.array([[-1]*w]*h)
init_board[1,2] = -2
#init_board[0,2] = -2

g_game = greedy_opponent(w, h, n, init_board, True)

print('Learning state values under random play.')
g_game.value_iteration(g_game.random_move, 20000)
g_game.temp_value = g_game.state_value
g_game.state_count =defaultdict(lambda: 0)

#print('Learning state values under greedy play.')
#g_game.value_iteration(g_game.greedy_move, 20000)
#g_game.temp_value = g_game.state_value


Learning state values under random play.


In [5]:
g_game.play_game()

Connect-3
(0)(0)(0)(1)(0)(0)
( ) .  .  1  . ( )
( ) .  .  #  . ( )
(0) 0  0  1  0 (0)
( )( )( )(1)( )( )
   ------------
    0  1  2  3
You won!


In [6]:
g_game.play_game(False)

Connect-3
(1)(1)(0)(0)(1)(1)
(0) .  .  .  0 ( )
(1) 1  0  #  1 (1)
(1) 1  0  0  1 (1)
(0)( )( )( )(0)( )
   ------------
    0  1  2  3
You won!
